# Comparison of occupancy analysis methods

How do different ways of summarizing occupancy by TOD and DOW compare in terms of their results and computational cost. The main things I want to compare are:

- hillmaker edge_bins setting 1 (fractional) vs 2 (whole bin)
- combinations of edge_bin and bin_size_minutes
- SQL snapshot approach using different grid sizes for epochs at which to do occupancy snapshots
- direct analysis of occupancy sample path using weighted statistics

Using datasets with varying LOS characteristics

- SSU - LOS on the order of a few to several hours
- SFCS - cycle share data from San Francisco. Highly variable ride durations with many short rides (median ~17 minutes).
- others?
  
Ideally I'd like to get to guidelines for:

- selecting a method and hyperparameter values based on the characteristics of the input data
- choosing an appropriate analysis date range to mitigate horizon and warmup effects based on the characteristics of the input data

## Random notes

In addition to the summary plots, look at the bydatetime plots.

## Preliminaries

In [2]:
# To auto-reload modules in jupyter notebook (so that changes in files *.py doesn't require manual reloading):
# https://stackoverflow.com/questions/5364050/reloading-submodules-in-ipython
%load_ext autoreload
%autoreload 2

Import commonly used libraries and magic command for inline plotting

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import hillmaker as hm


In [4]:
%matplotlib inline

## The stops datasets

We'll use two different datasets.

### ssu_2024.csv

This is based on the orginal ShortStay.csv file but with more realistic LOS distributions. The arrival timestamps are identical. See **generate_ssu_stops.ipynb** for how the file was generated. These LOS distributions have means ranging from 2 - 8 hours and are all Erlang distributions with different numbers of stages.

### sfcs_trip.csv

San Francisco cycle share data. The rental durations are relatively short with the median around 17 minutes. Of course, there's lots of variability.

In [5]:
ssu_stopdata = './data/ssu_2024.csv'
sfcs_stopdata = './data/sfcs_trip.csv'

ssu_2024 = pd.read_csv(ssu_stopdata, parse_dates=['InRoomTS','OutRoomTS'])

sfcs_trip = pd.read_csv(sfcs_stopdata, parse_dates=['start_date','end_date'])
sfcs_trip['start_date'] = sfcs_trip['start_date'].dt.tz_localize(None)
sfcs_trip['end_date'] = sfcs_trip['end_date'].dt.tz_localize(None)



## Initial hillmaker runs

I'll run these with default settings (edge_bins=1, and bin_size_minutes=60). We can use this run to explore the LOS summaries.

### SSU

In [6]:
ssu_2024.head()

,PatID,InRoomTS,OutRoomTS,PatType,LOS_hours
0,1,2024-01-01 07:44:00,2024-01-01 09:20:00,IVT,1.600000
1,2,2024-01-01 08:28:00,2024-01-01 11:13:00,IVT,2.750000
2,3,2024-01-01 11:44:00,2024-01-01 12:48:00,MYE,1.066667
3,4,2024-01-01 11:51:00,2024-01-01 21:10:00,CAT,9.316667
4,5,2024-01-01 12:10:00,2024-01-01 12:57:00,IVT,0.783333


For the following, I'm not exporting any csv or plots. I've got Python scripts (see `run_ssu_2024.py`) that do the actual production runs over a grid of values of `edge_bins`, `bin_size_minutes`, and `highres_bin_size_minutes`.

The main initial findings after snooping some of the results are:

- for `edge_bins=1` the occupancy summaries are not impacted much at all by the use of `highres_bin_size_minutes`. Mean and p95 occupancy for a given setting of `bin_size_minutes` is identical across all values of `highres_bin_size_minutes`.
- for `edge_bins=2` it is critical to use a small value of for `highres_bin_size_minutes` to avoid extreme overestimation of occupancy (both mean and p95).
- even relatively coarse bin sizes are fine as long as `edge_bins=1`.

**QUESTION** Should we get rid of `edge_bins=2`?

- avoids user doing something silly
- the addition of the high resolution bin size parameter gives the ability to compute very accurate occupancy statistics and still be able to roll them up and report at more reasonable bin sizes.

**QUESTION** How are percentiles based on rolled up occupancy different than percentiles computed on the higher resolution bin sizes? In other words, what does e1_b5_r5 look like in terms of percentiles?

**QUESTION** How does hillmaker approach differ from a weighted sample occupancy path approach?

**QUESTION** How does hillmaker approach differ from doing a SQL based snapshot at fixed time points?

In [7]:
# Scenario inputs

ssu_e1_b60_r5_dict = {'scenario_name': 'ssu_edge1_bin60_res5',
                   'stops_df': ssu_2024,
                   'in_field': 'InRoomTS',
                   'out_field': 'OutRoomTS',
                   'start_analysis_dt': '2024-01-01',
                   'end_analysis_dt': '2024-03-30',
                   'cat_field': 'PatType',
                   'output_path': 'output/ssu_2024',
                   'bin_size_minutes': 60,
                   'highres_bin_size_minutes': 5,
                   'keep_highres_bydatetime': True,
                   'edge_bins': 1,
                   'export_bydatetime_csv': False,
                   'export_summaries_csv': False,
                   'make_all_dow_plots': False,
                   'make_all_week_plots': False,
                   'export_all_dow_plots': False,
                   'export_all_week_plots': False,
                   'verbosity': 1,
                  }
    
ssu_e1_b60_r5 = hm.Scenario(**ssu_e1_b60_r5_dict)

ssu_e1_b60_r5.make_hills()

2023-09-26 12:58:08,547 - hillmaker.hills - INFO - Starting scenario ssu_edge1_bin60_res5 at 8.099541874
2023-09-26 12:58:08,552 - hillmaker.bydatetime - INFO - min of intime: 2024-01-01 07:44:00
2023-09-26 12:58:08,555 - hillmaker.bydatetime - INFO - max of intime: 2024-03-30 14:46:00
2023-09-26 12:58:08,557 - hillmaker.bydatetime - INFO - min of outtime: 2024-01-01 09:20:00
2023-09-26 12:58:08,559 - hillmaker.bydatetime - INFO - max of outtime: 2024-03-31 00:36:00
2023-09-26 12:58:08,562 - hillmaker.bydatetime - INFO - start analysis: 2024-01-01, end analysis: 2024-03-30
2023-09-26 12:58:08,585 - hillmaker.bydatetime - INFO - min of entry time_bin = 92
2023-09-26 12:58:08,588 - hillmaker.bydatetime - INFO - max of exit time_bin = 25809 and num_bins=25920
2023-09-26 12:58:08,825 - hillmaker.bydatetime - INFO - cat IVT {'inner': 10911}
2023-09-26 12:58:08,842 - hillmaker.bydatetime - INFO - cat IVT num_arrivals_hm 10911 num_arrivals_stops 10911
2023-09-26 12:58:08,843 - hillmaker.bydat

In [8]:
ssu_e1_b60_r5.hills.keys()

dict_keys(['bydatetime', 'summaries', 'length_of_stay', 'settings', 'bydatetime_highres', 'plots', 'runtime'])

In [9]:
ssu_e1_b60_r5.hills['runtime']

11.597733539

In [ ]:
# Scenario inputs

ssu_e1_b60_r60_dict = {'scenario_name': 'ssu_edge1_bin60_res60',
                   'stops_df': ssu_2024,
                   'in_field': 'InRoomTS',
                   'out_field': 'OutRoomTS',
                   'start_analysis_dt': '2024-01-01',
                   'end_analysis_dt': '2024-03-30',
                   'cat_field': 'PatType',
                   'output_path': 'output/ssu_2024',
                   'bin_size_minutes': 60,
                   'highres_bin_size_minutes': 60,
                   'keep_highres_bydatetime': True,
                   'edge_bins': 1,
                   'export_bydatetime_csv': False,
                   'export_summaries_csv': False,
                   'make_all_dow_plots': False,
                   'make_all_week_plots': False,
                   'export_all_dow_plots': False,
                   'export_all_week_plots': False,
                   'verbosity': 1,
                  }
    
ssu_e1_b60_r60 = hm.Scenario(**ssu_e1_b60_r60_dict)

ssu_e1_b60_r60.make_hills()

## SFCS

In [ ]:
sfcs_trip.head()

In [ ]:
sfcs_trip_noparse.head()

In [ ]:
sfcs_trip['duration'].describe()

In [ ]:
1107/60

With such a short mean service time, I'd expect 15 minute bins to show considerable difference for different edge_bins settings.

In [ ]:
# Scenario inputs

sfcs_e1_b15_dict = {'scenario_name': 'sfcs_edge1_bin15',
                   'stops_df': sfcs_trip,
                   'in_field': 'start_date',
                   'out_field': 'end_date',
                   'start_analysis_dt': '2014-05-01',
                   'end_analysis_dt': '2014-09-30',
                   'cat_field': 'subscription_type',
                   'output_path': 'output/sfcs_trip',
                   'bin_size_minutes': 15,
                   'edge_bins': 1,
                   'export_bydatetime_csv': True,
                   'export_summaries_csv': True,
                   'make_all_dow_plots': True,
                   'make_all_week_plots': True,
                   'export_all_dow_plots': True,
                   'export_all_week_plots': True,
                   'verbosity': 1,
                  }
    
sfcs_e1_b15 = hm.Scenario(**sfcs_e1_b15_dict)


sfcs_e1_b15.make_hills()

In [ ]:
# Scenario inputs

sfcs_e2_b15_dict = {'scenario_name': 'sfcs_edge2_bin15',
                   'stops_df': sfcs_trip,
                   'in_field': 'start_date',
                   'out_field': 'end_date',
                   'start_analysis_dt': '2014-05-01',
                   'end_analysis_dt': '2014-09-30',
                   'cat_field': 'subscription_type',
                   'output_path': 'output/sfcs_trip',
                   'bin_size_minutes': 15,
                   'edge_bins': 2,
                   'export_bydatetime_csv': True,
                   'export_summaries_csv': True,
                   'make_all_dow_plots': True,
                   'make_all_week_plots': True,
                   'export_all_dow_plots': True,
                   'export_all_week_plots': True,
                   'verbosity': 1,
                  }
    
sfcs_e2_b15 = hm.Scenario(**sfcs_e2_b15_dict)


sfcs_e2_b15.make_hills()